In [1]:
import sys
sys.path.append("../")
import pandas as pd
import openpyxl as oxl

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model,should_use_reasoning_model
from IPython.display import display, Markdown
from utils.data_loader import load_and_clean_data
from utils.token_utils import count_messages_tokens,fit_within_context

### Load data from text fiie

In [2]:
messages = load_and_clean_data("../data/raw/Incidents.txt")
messages

'ID: 1, Time: 08:00 AM, Area: Gampaha, People: 4, Ages: 20-40, Main Need: Water, Message: Thirsty but safe on roof. Water level stable.\nID: 2, Time: 08:15 AM, Area: Ja-Ela, People: 1, Ages: 75, Main Need: Insulin, Message: Diabetic, missed dose yesterday. Feeling faint.\nID: 3, Time: 08:20 AM, Area: Ragama, People: 2, Ages: 10, 35, Main Need: Rescue, Message: Water approaching neck level. Child is crying.'

### Checking long chain-spam messages & apply the logic


In [3]:
messages_response = [
    {"role": "user", "content": messages}
]

# Count tokens
token_info = count_messages_tokens(
    messages=messages_response,
    provider="groq",
    model="llama-3.1-8b-instant"
)

# Enforce budget rule
if token_info["estimated_total"] > 150:
    print(messages)
    print("\nTotal tokens:", token_info["estimated_total"])
    print("\nBLOCKED/TRUNCATED")
else:
    print("Message OK:")
    print("\n" + messages)

ID: 1, Time: 08:00 AM, Area: Gampaha, People: 4, Ages: 20-40, Main Need: Water, Message: Thirsty but safe on roof. Water level stable.
ID: 2, Time: 08:15 AM, Area: Ja-Ela, People: 1, Ages: 75, Main Need: Insulin, Message: Diabetic, missed dose yesterday. Feeling faint.
ID: 3, Time: 08:20 AM, Area: Ragama, People: 2, Ages: 10, 35, Main Need: Rescue, Message: Water approaching neck level. Child is crying.

Total tokens: 151

BLOCKED/TRUNCATED
